# Banana Leaf Disease Classifier

In [1]:
!pip install lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00


In [3]:
# Unzip dataset

In [4]:
!gdown 1dXhALACGB0uAd9NanwmRnZl5xizExvFQ

Downloading...
From: https://drive.google.com/uc?id=1dXhALACGB0uAd9NanwmRnZl5xizExvFQ
To: /content/processed.zip
100% 7.03M/7.03M [00:00<00:00, 16.4MB/s]


In [15]:
!unzip -q -o processed.zip

In [6]:
# Dataset Helper Functions

In [1]:
import glob
import os
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import cv2
from pathlib import Path

In [2]:
class BananaLeafDiseaseDataset(Dataset):
    def __init__(self, target_dir: str, transform=None):
        self.transform = transform

        # Load image paths
        self.image_paths = list(Path(target_dir).glob('*/*.jpg'))
        self.classes, self.class_to_idx = self.load_classes(target_dir)

    def load_image(self, idx: int):
        return Image.open(self.image_paths[idx])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = self.load_image(idx)
        class_name = self.image_paths[idx].parent.name
        class_idx = self.class_to_idx[class_name]

        if self.transform:
            return self.transform(img), class_idx
        else:
            return img, class_idx


    def load_classes(self, target_dir):
        """
            Returns:
                classes[]
                class_to_idx{}
        """
        classes = sorted(entry.name for entry in os.scandir(target_dir) if entry.is_dir())
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

In [3]:
basic_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((144,144)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(degrees = 45),
    transforms.RandomHorizontalFlip(p = 0.25),
    transforms.Resize((144,144)),
    transforms.ColorJitter(brightness=0.5, contrast=1, saturation=0.1, hue=0.5),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [4]:
train_dataset = BananaLeafDiseaseDataset('./processed/train', transform=train_transforms)
val_dataset = BananaLeafDiseaseDataset('./processed/val', transform=basic_transforms)
test_dataset = BananaLeafDiseaseDataset('./processed/test', transform=basic_transforms)

In [5]:
# train_size = 0.8
# test_size = 1 - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [6]:
# train_dataset.dataset.transform = train_transforms
# test_dataset.dataset.transform = basic_transforms

In [7]:
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=True)

In [ ]:
# CustomLogger

In [8]:
class CustomLogger():
  def log():
    pass

In [9]:
import lightning as pl
import torch
import numpy as np
from torch import nn
from einops.layers.torch import Rearrange
import torch.nn.functional as F

In [10]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

In [21]:
class MixerBlock(nn.Module):
    def __init__(self, dim, num_patch, token_dim, channel_dim, dropout = 0.):
        super().__init__()

        self.token_mix = nn.Sequential(
            nn.LayerNorm(dim),
            Rearrange('b n d -> b d n'),
            FeedForward(num_patch, token_dim, dropout),
            Rearrange('b d n -> b n d')
        )

        self.channel_mix = nn.Sequential(
            nn.LayerNorm(dim),
            FeedForward(dim, channel_dim, dropout),
        )

    def forward(self, x):
        x = x + self.token_mix(x)
        x = x + self.channel_mix(x)
        return x

class MLPMixer(pl.LightningModule):
    def __init__(
        self,
        in_channels,
        dim,
        num_classes,
        patch_size,
        image_size,
        depth,
        token_dim,
        channel_dim
    ):
        super().__init__()

        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        self.num_patch =  (image_size// patch_size) ** 2
        self.to_patch_embedding = nn.Sequential(
            nn.Conv2d(in_channels, dim, patch_size, patch_size),
            Rearrange('b c h w -> b (h w) c'),
        )

        self.mixer_blocks = nn.ModuleList([])

        for _ in range(depth):
            self.mixer_blocks.append(MixerBlock(dim, self.num_patch, token_dim, channel_dim))

        self.layer_norm = nn.LayerNorm(dim)

        self.mlp_head = nn.Sequential(
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        x = self.to_patch_embedding(x)
        for mixer_block in self.mixer_blocks:
            x = mixer_block(x)

        x = self.layer_norm(x)
        x = x.mean(dim=1)
        return self.mlp_head(x)


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4)
        return optimizer

    def _calculate_loss(self, batch, mode="train"):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=-1) == y).float().mean()

        self.log("%s_loss" % mode, loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        self.log("%s_acc" % mode, acc, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="train")

    def validation_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="test")

In [25]:
model = MLPMixer(in_channels=3, image_size=224, patch_size=16, num_classes=3,
                     dim=192, depth=6, token_dim=96, channel_dim=512)

In [24]:
from lightning.pytorch.loggers import CSVLogger
logger = CSVLogger("logs", name="Mixer-192-6-96-512")

In [19]:
trainer = pl.Trainer(max_epochs=50, logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Missing logger folder: logs/Mixer-192-6-96-512

  | Name               | Type       | Params
--------------------------------------------------
0 | to_patch_embedding | Sequential | 37.1 K
1 | mixer_blocks       | ModuleList | 1.6 M 
2 | layer_norm         | LayerNorm  | 384   
3 | mlp_head           | Sequential | 579   
--------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.409     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.test(model, dataloaders=test_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.6964980363845825     │
│      test_loss_epoch      │    0.6395615339279175     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.6395615339279175, 'test_acc_epoch': 0.6964980363845825}]

In [ ]:
!zip -r lightning_logs.zip lightning_logs

In [ ]:
trainer.callback_metrics

{}